# Built-in Neural Networks in Φ<sub>Flow</sub>

Apart from being a general purpose differential physics library, Φ<sub>Flow</sub> also provides a number of backend-agnostic way of setting up optimizers and some neural networks.

The following network architectures are supported:

* Fully-connected networks: `dense_net()`
* Convolutional networks: `conv_net()`
* Residual networks: `res_net()`
* U-Nets: `u_net()`
* Convolutional classifiers: `conv_classifier()`

In addition to zero-padding, the convolutional neural networks all support circular periodic padding across feature map spatial dimensions to maintain periodicity.

All network-related convenience functions in Φ<sub>Flow</sub> support PyTorch, TensorFlow and Jax/Stax, setting up native networks.


In [1]:
from phi.tf.flow import *
from phi.jax.stax.flow import *
from phi.torch.flow import *

## Fully-connected Networks
Fully-connected neural networks are available in Φ<sub>Flow</sub> via `dense_net()`.

#### Arguments

* `in_channels` : size of input layer, int
* `out_channels` = size of output layer, int
* `layers` : tuple of linear layers between input and output neurons, list or tuple
* `activation` : activation function used within the layers, string
* `batch_norm` : use of batch norm after each linear layer, bool

In [2]:
net = dense_net(in_channels=1, out_channels=1, layers=[8, 8], activation='ReLU', batch_norm=False)  # Implemented for PyTorch, TensorFlow, Jax-Stax)

In [3]:
optimizer = adam(net, 1e-3)
BATCH = batch(batch=100)

def loss_function(data: Tensor):
    prediction = math.native_call(net, data)
    label = math.sin(data)
    return math.l2_loss(prediction - label), data, label

print(f"Initial loss: {loss_function(math.random_normal(BATCH))[0]}")
for i in range(100):
    loss, _data, _label = update_weights(net, optimizer, loss_function, data=math.random_normal(BATCH))
print(f"Final loss: {loss}")

Initial loss: (batchᵇ=100) 0.246 ± 0.178 (2e-04...6e-01)
Final loss: (batchᵇ=100) 0.087 ± 0.073 (4e-04...3e-01)


## U-Nets
Φ<sub>Flow</sub> provides a built in U-net architecture, classically popular for Semantic Segmentation in Computer Vision, composed of downsampling and upsampling layers.

 #### Arguments

* `in_channels`: input channels of the feature map, dtype : int
* `out_channels` : output channels of the feature map, dtype : int
* `levels` : number of levels of down-sampling and upsampling, dtype : int
* `filters` : filter sizes at each down/up sampling convolutional layer, if the input is integer all conv layers have the same filter size,<br> dtype : int or tuple
* `activation` : activation function used within the layers, dtype : string
* `batch_norm` : use of batchnorm after each conv layer, dtype : bool
* `in_spatial` : spatial dimensions of the input feature map, dtype : int
* `use_res_blocks` : use convolutional blocks with skip connections instead of regular convolutional blocks, dtype : bool
* `**kwargs` : placeholder for arguments not supported by the function (such as layers in res_net and conv_net)

In [4]:
net = u_net(in_channels= 1, out_channels= 2, levels=4, filters=16, batch_norm=True, activation='ReLU', in_spatial=2, use_res_blocks=False)

In [5]:
#Loss function for training in the network to identify noise parameters
def loss_function(scale: Tensor, smoothness: Tensor):
    grid = CenteredGrid(Noise(scale=scale, smoothness=smoothness), x=64, y=64)

    print(f'Grid Shape : {grid.shape}')
    pred_scale, pred_smoothness = field.native_call(net, grid).vector
    return math.l2_loss(pred_scale - scale) + math.l2_loss(pred_smoothness - smoothness)

In [6]:
optimizer = adam(net, learning_rate=1e-3)
gt_scale = math.random_uniform(batch(examples=50), low=1, high=10)
gt_smoothness = math.random_uniform(batch(examples=50), low=.5, high=3)

print(f"Initial loss: {loss_function(gt_scale, gt_smoothness)}")
for i in range(10):
    loss = update_weights(net, optimizer, loss_function, gt_scale, gt_smoothness)
    print(f'Iter : {i}, Loss : {loss}')
print(f"Final loss: {loss}")

Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Initial loss: (examplesᵇ=50) 8.28e+04 ± 5.6e+04 (8e+03...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 0, Loss : (examplesᵇ=50) 8.28e+04 ± 5.6e+04 (8e+03...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 1, Loss : (examplesᵇ=50) 8.09e+04 ± 5.3e+04 (9e+03...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 2, Loss : (examplesᵇ=50) 7.95e+04 ± 5.1e+04 (1e+04...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 3, Loss : (examplesᵇ=50) 7.85e+04 ± 5.0e+04 (1e+04...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 4, Loss : (examplesᵇ=50) 7.74e+04 ± 4.8e+04 (1e+04...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 5, Loss : (examplesᵇ=50) 7.68e+04 ± 4.7e+04 (8e+03...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 6, Loss : (examplesᵇ=50) 7.60e+04 ± 4.7e+04 (9e+03...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 7, Loss : (examplesᵇ=50) 7.55e+04 ± 4.6e+04 (1e+04...2e+05)
Grid Shape : (examples

## Convolutional Networks
Built in Conv-Nets are also provided. Contrary to the classical convolutional neural networks, the feature map spatial size remains the same throughout the layers.
Each layer of the network is essentially a convolutional block comprising of two conv layers.
A filter size of 3 is used in the convolutional layers.

#### Arguments

* `in_channels` : input channels of the feature map, dtype : int
* `out_channels` : output channels of the feature map, dtype : int <br>
* `layers` : list or tuple of output channels for each intermediate layer between the input and final output channels, dtype : list or tuple <br>
* `activation` : activation function used within the layers, dtype : string <br>
* `batch_norm` : use of batchnorm after each conv layer, dtype : bool <br>
* `in_spatial` : spatial dimensions of the input feature map, dtype : int <br>
* `**kwargs` : placeholder for arguments not supported by the function


In [7]:
net = conv_net(in_channels=1, out_channels=2, layers=[2,4,4,2], activation='ReLU', batch_norm=True, in_spatial=2)

In [8]:
optimizer = adam(net, learning_rate=1e-3)
gt_scale = math.random_uniform(batch(examples=50), low=1, high=10)
gt_smoothness = math.random_uniform(batch(examples=50), low=.5, high=3)

print(f"Initial loss: {loss_function(gt_scale, gt_smoothness)}")
for i in range(10):
    loss = update_weights(net, optimizer, loss_function, gt_scale, gt_smoothness)
    print(f'Iter : {i}, Loss : {loss}')
print(f"Final loss: {loss}")

Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Initial loss: (examplesᵇ=50) 8.21e+04 ± 4.9e+04 (1e+04...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 0, Loss : (examplesᵇ=50) 8.21e+04 ± 4.9e+04 (1e+04...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 1, Loss : (examplesᵇ=50) 8.20e+04 ± 4.9e+04 (1e+04...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 2, Loss : (examplesᵇ=50) 8.18e+04 ± 4.9e+04 (1e+04...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 3, Loss : (examplesᵇ=50) 8.17e+04 ± 4.9e+04 (1e+04...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 4, Loss : (examplesᵇ=50) 8.16e+04 ± 4.9e+04 (1e+04...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 5, Loss : (examplesᵇ=50) 8.15e+04 ± 4.9e+04 (1e+04...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 6, Loss : (examplesᵇ=50) 8.14e+04 ± 4.9e+04 (1e+04...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 7, Loss : (examplesᵇ=50) 8.12e+04 ± 4.9e+04 (1e+04...2e+05)
Grid Shape : (examples

## Residual Networks
Built in Res-Nets are provided in the Φ<sub>Flow</sub> framework. Similar to the conv-net, the feature map spatial size remains the same throughout the layers.<br>These networks use residual blocks composed of two conv layers with a skip connection added from the input to the output feature map.<br> A default filter size of 3 is used in the convolutional layers.<br><br>

#### Arguments

* `in_channels` : input channels of the feature map, dtype : int
* `out_channels` : output channels of the feature map, dtype : int
* `layers` : list or tuple of output channels for each intermediate layer between the input and final output channels, dtype : list or tuple
* `activation` : activation function used within the layers, dtype : string
* `batch_norm` : use of batchnorm after each conv layer, dtype : bool
* `in_spatial` : spatial dimensions of the input feature map, dtype : int
* `**kwargs` : placeholder for arguments not supported by the function

In [9]:
net = res_net(in_channels=1, out_channels=2, layers=[2,4,4,2], activation='ReLU', batch_norm=True, in_spatial=2)

In [10]:
optimizer = adam(net, learning_rate=1e-3)
gt_scale = math.random_uniform(batch(examples=50), low=1, high=10)
gt_smoothness = math.random_uniform(batch(examples=50), low=.5, high=3)

print(f"Initial loss: {loss_function(gt_scale, gt_smoothness)}")
for i in range(10):
    loss = update_weights(net, optimizer, loss_function, gt_scale, gt_smoothness)
    print(f'Iter : {i}, Loss : {loss}')
print(f"Final loss: {loss}")

Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Initial loss: (examplesᵇ=50) 9.13e+04 ± 5.6e+04 (3e+04...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 0, Loss : (examplesᵇ=50) 9.13e+04 ± 5.6e+04 (3e+04...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 1, Loss : (examplesᵇ=50) 9.10e+04 ± 5.6e+04 (3e+04...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 2, Loss : (examplesᵇ=50) 9.08e+04 ± 5.6e+04 (3e+04...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 3, Loss : (examplesᵇ=50) 9.05e+04 ± 5.6e+04 (3e+04...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 4, Loss : (examplesᵇ=50) 9.04e+04 ± 5.6e+04 (3e+04...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 5, Loss : (examplesᵇ=50) 9.01e+04 ± 5.5e+04 (3e+04...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 6, Loss : (examplesᵇ=50) 9.00e+04 ± 5.6e+04 (2e+04...2e+05)
Grid Shape : (examplesᵇ=50, xˢ=64, yˢ=64)
Iter : 7, Loss : (examplesᵇ=50) 8.97e+04 ± 5.5e+04 (3e+04...2e+05)
Grid Shape : (examples

## Invertible Nets

Phiflow also provides invertible neural networks that are capable of inverting the output tensor back to the input tensor initially passed.\
These networks have far reaching applications in predicting input parameters of a problem given its observations.\
Invertible nets are composed of multiple concatenated coupling blocks wherein each such block consists of arbitrary neural networks.
 
Currently these arbitrary neural networks could be set as u_net(default), conv_net, res_net or dense_net blocks with in_channels = out_channels. The architecture used is popularized by [Real NVP](https://arxiv.org/abs/1605.08803).

### Arguments
* `in_channels` : input channels of the feature map, dtype : int
* `num_blocks` : number of coupling blocks inside the invertible net, dtype : int
* `activation` : activation function used within the layers, dtype : string
* `batch_norm` : use of batchnorm after each layer, dtype : bool
* `in_spatial` : spatial dimensions of the input feature map, dtype : int
* `net` : type of neural network blocks used in coupling layers, dtype : str
* `**kwargs` : placeholder for arguments not supported by the function

Note: Currently supported values for net are 'u_net'(default), 'conv_net' and 'res_net'. For choosing 'dense_net' as the network block in coupling layers in_spatial must be set to zero.

In [11]:

def loss_function_inn(grid, scale: Tensor, smoothness: Tensor):
    
    
    pred_scale, pred_smoothness = field.native_call(net, grid).vector
    pred_scale = math.expand(pred_scale, channel(c=1))
    pred_smoothness = math.expand(pred_smoothness, channel(c=1))
    output_grid = math.concat((pred_scale, pred_smoothness), dim='c')
    
    return math.l2_loss(pred_scale - scale) + math.l2_loss(pred_smoothness - smoothness), output_grid
    

In [12]:
net = invertible_net(in_channels=2, num_blocks=2, activation='ReLU', batch_norm=True, in_spatial=2, net='u_net')

In [13]:
optimizer = adam(net, learning_rate=1e-3)
gt_scale = math.random_uniform(batch(examples=50), low=1, high=10)
gt_smoothness = math.random_uniform(batch(examples=50), low=.5, high=3)

input_grid = CenteredGrid(Noise(scale=gt_scale, smoothness=gt_smoothness), x=32, y=32)
    
# Expanding channels to 2 by repeating it along channel dimension 
# in order to obtain feature maps for both pred_scale and pred_smoothness (in_channels = out_channels = 2)
input_grid = math.expand(input_grid, channel(c=2))

for i in range(10):
    loss, grid = update_weights(net, optimizer, loss_function_inn, input_grid, gt_scale, gt_smoothness)
    print(f'Iter : {i}, Loss : {loss}')
    
print(f"Final loss: {loss}")

Iter : 0, Loss : (examplesᵇ=50) 2.06e+04 ± 1.3e+04 (4e+03...5e+04)
Iter : 1, Loss : (examplesᵇ=50) 1.77e+04 ± 1.2e+04 (3e+03...4e+04)
Iter : 2, Loss : (examplesᵇ=50) 1.56e+04 ± 1.1e+04 (3e+03...4e+04)
Iter : 3, Loss : (examplesᵇ=50) 1.42e+04 ± 9.9e+03 (3e+03...4e+04)
Iter : 4, Loss : (examplesᵇ=50) 1.30e+04 ± 9.1e+03 (3e+03...4e+04)
Iter : 5, Loss : (examplesᵇ=50) 1.19e+04 ± 8.2e+03 (3e+03...3e+04)
Iter : 6, Loss : (examplesᵇ=50) 1.09e+04 ± 7.4e+03 (3e+03...3e+04)
Iter : 7, Loss : (examplesᵇ=50) 1.00e+04 ± 6.6e+03 (3e+03...3e+04)
Iter : 8, Loss : (examplesᵇ=50) 9.18e+03 ± 5.9e+03 (3e+03...2e+04)
Iter : 9, Loss : (examplesᵇ=50) 8.43e+03 ± 5.3e+03 (3e+03...2e+04)
Final loss: (examplesᵇ=50) 8.43e+03 ± 5.3e+03 (3e+03...2e+04)


Original input feature map can be obtained by passing the predicted feature map once again through the n

In [14]:

grid = field.native_call(net, input_grid, False)
reconstructed_input =  field.native_call(net, grid, True) # invert = True   
print('Loss between initial input and prediction',math.l2_loss(input_grid - grid))
print('Loss between initial input and reconstructed input',math.l2_loss(input_grid - reconstructed_input))

Loss between initial input and prediction (examplesᵇ=50) 8.82e+03 ± 6.9e+03 (7e+02...2e+04)
Loss between initial input and reconstructed input (examplesᵇ=50) 4.84e-09 ± 2.3e-09 (2e-09...1e-08)
